In [1]:
#Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Import Bokeh
import bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LogColorMapper, ColumnDataSource, HoverTool, LinearColorMapper, ColorBar, Panel, CustomJS, DataRange1d
from bokeh.palettes import Viridis6, Spectral6, Category10, Spectral, Viridis5, Dark2
from bokeh.models.widgets import Select, Slider, Tabs, Select, Button, Dropdown
from bokeh.layouts import column, row, WidgetBox
from bokeh.palettes import Oranges
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.transform import factor_cmap
#Import UMAP
from sklearn.model_selection import train_test_split
import umap
import random
#Magic commands
bokeh.io.output_notebook()
%matplotlib inline

Loading BokehJS ...

In [2]:
from immunova.data.project import Project
from immunova.data.mongo_setup import global_init
from immunova.flow.gating.actions import Template, Gating
from tqdm import tqdm_notebook, tqdm
from functools import partial
import warnings
warnings.filterwarnings('ignore')
global_init()

In [3]:
iltis = Project.objects(project_id='ILTIS').get()
t1_exp = iltis.load_experiment('t1_exp')

In [4]:
t_template = Template(experiment=t1_exp, sample_id='sep13')

In [19]:
def UMAP_Dashboard(app):
    def create_umap(id_):
        print("Loading dataset...")
        t_template = Template(experiment=t1_exp, sample_id=id_)
        cd3 = t_template.get_population_df('cd3+')
        cd3['Population'] = 'None'
        gdt = t_template.populations['gdt']['index']
        mait = t_template.populations['mait']['index']
        cd4 = t_template.populations['cd4']['index']
        cd8 = t_template.populations['cd8']['index']
        for name, idx in zip(['gdt', 'mait', 'cd4', 'cd8'], [gdt, mait, cd4, cd8]):
            cd3['Population'] = cd3['Population'].mask(cd3.index.isin(idx), name)
        features = [c for c in cd3.columns if c not in ['Time', 'Population', 'umap0', 'umap1']]
        features = [f for f in features if f.find('SSC') == -1 and f.find('FSC') == -1]
        print("Generating UMAP...")
        reducer = umap.UMAP()
        embedding = reducer.fit_transform(cd3[features])
        cd3["umap0"] = embedding[:, 0]
        cd3["umap1"] = embedding[:, 1]
        plot_data = dict(x=cd3["umap0"].values.tolist(),
                        y=cd3["umap1"].values.tolist(),
                        c=cd3["Population"].values.tolist(),
                        index=cd3.index.tolist())
        return(ColumnDataSource(plot_data), cd3)

    def create_facs_data(indicies ,all_data, x_param, y_param):
        x = all_data[x_param][indicies]
        y = all_data[y_param][indicies]
        pop = all_data["Population"][indicies]
        plot_data = dict(x=x, y=y, c=pop)
        return(ColumnDataSource(plot_data))

    def build_umap_plot(src):
        TOOLS = "pan,wheel_zoom,reset,save,lasso_select"
        umap_scatter = figure(plot_width=450, plot_height=450, tools=TOOLS, title="UMAP")
        factors = list(set(src.data['c']))
        umap_scatter.circle('x', 'y', color=factor_cmap('c', palette=Dark2[5], factors=factors), size=0.5, 
                            source=src, legend='c')
        umap_scatter.legend.orientation = "horizontal"
        umap_scatter.legend.location = "top_center"
        return(umap_scatter)

    def build_facs_plot(src):
        TOOLS = "pan,wheel_zoom,reset,save"
        x_max = max(src.data['x'])
        y_max = max(src.data['y'])
        factors = src.data['c'].unique().tolist()
        scatter = figure(plot_width=500, plot_height=500, title="FACS Plot", 
                         x_range=(0, x_max), y_range=(0, y_max), tools=TOOLS)
        scatter.circle('x','y',source=src,color=factor_cmap('c', palette=Dark2[5], factors=factors),legend='c', size=0.5)
        scatter.xaxis.axis_label = "FSC-A"
        scatter.yaxis.axis_label = "SSC-A"
        scatter.legend.orientation = "horizontal"
        scatter.legend.location = "top_center"
        return(scatter)

    def update_facs_plot(event):
        indices = umap_src.selected.indices
        x_param = x_dd.value
        y_param = y_dd.value
        new_data = create_facs_data(indices, all_data, x_param, y_param)
        facs_plot.xaxis.axis_label = x_param
        facs_plot.yaxis.axis_label = y_param
        facs_plot.x_range.start = 0
        facs_plot.x_range.end = max(new_data.data['x'])
        facs_plot.y_range.start = 0
        facs_plot.y_range.end = max(new_data.data['y'])
        facs_src.data.update(new_data.data)
        

    umap_src, all_data = create_umap('sep13')
    facs_src = create_facs_data(all_data.index.tolist(), all_data, "FSC-A", "SSC-A")

    #Widgets
    menu = [x for x in all_data.columns.tolist() if x not in ['Time', 'umap0', 'umap1', 'Population']]
    menu.append("FSC-A")
    menu.append("SSC-A")
    x_dd = Select(title="FACS X-Axis",value="FSC-A", options=menu)
    y_dd = Select(title="FACS Y-Axis",value="SSC-A", options=menu)
    b = Button(label="Plot Selection", button_type="success")
    
    b.on_click(update_facs_plot)
    #Init plot and set layout
    controls = WidgetBox(x_dd, y_dd, b)
    umap_plot = build_umap_plot(umap_src)
    facs_plot = build_facs_plot(facs_src)
    layout=row(umap_plot, facs_plot, controls)
    app.add_root(layout)

In [20]:
dashboard_handler = FunctionHandler(UMAP_Dashboard)
dashboard_app = Application(dashboard_handler)
show(dashboard_app)

Loading dataset...
Generating UMAP...


ERROR:bokeh.server.protocol_handler:error handling message Message 'EVENT' (revision 1) content: '{"event_name":"button_click","event_values":{"model_id":"1433"}}': ValueError('Out of range float values are not JSON compliant')
